# Imitation Learning - Lab 01

O objetivo desta laboratório é experimentar a aprendizagem por imitação (imitation learning), em que o modelo (rede) aprende a imitar as ações de um especialista (humano). No lugar de experiências coletadas de um especialista humano, aqui as demonstrações serão fornecidas por meio de uma política de especialistas que treinamos para você. 

- Usaremos um tipo de aprendizado por imitação, conhecido como clonagem comportamental (behavioral cloning). Isso significa que treinaremos nossa rede de forma supervisionada.
- A saída da rede é a política de direção, representada pelo ângulo de direção desejado e/ou aceleração ou frenagem. Por exemplo, podemos ter um neurônio de saída de regressão para o ângulo de direção e um neurônio para aceleração ou frenagem (já que não podemos ter os dois ao mesmo tempo).
- A entrada da rede pode ser:
Dados brutos do sensor. Por exemplo, uma imagem da câmera. 
- Criaremos o conjunto de dados de treinamento com a ajuda do especialista. Em cada etapa da jornada, iremos registrar:
    - O estado atual do ambiente. Estes podem ser os dados brutos do sensor ou a representação da vista de cima para baixo. Usaremos o estado atual como entrada para o modelo.
    - As ações do especialista no estado atual do ambiente (ângulo de direção, freio / aceleração). Esses serão os dados de destino da rede. Durante o treinamento, vamos minimizar o erro entre as previsões da rede e as ações usando gradient descent. Desta forma, ensinaremos a rede a imitar o especialista.

<br>

A seguir está uma ilustração do cenário de Behavioral Cloning:

<br>

<img src='https://drive.google.com/uc?id=1ozI1x1hNgIa_IXNsxUm4V-YFADXlKxus' width="600" height="400">


## Configuração


Você precisará fazer uma cópia deste notebook em seu Google Drive antes de editar. Você pode fazer isso com **Arquivo → Salvar uma cópia no Drive**.

In [1]:
import os
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
# Seu trabalho será armazenado em uma pasta chamada `minicurso_rl` por padrão 
# para evitar que o tempo limite da instância do Colab exclua suas edições

DRIVE_PATH = "/content/gdrive/My\ Drive/minicurso_rl"
DRIVE_PYTHON_PATH = DRIVE_PATH.replace("\\", "")
if not os.path.exists(DRIVE_PYTHON_PATH):
  %mkdir $DRIVE_PATH

SYM_PATH = "/content/minicurso_rl"
if not os.path.exists(SYM_PATH):
  !ln -s $DRIVE_PATH $SYM_PATH

Instalando as dependências

In [3]:
!pip install -U cloudpickle > /dev/null 2>&1 
!pip install "gym[all]" > /dev/null 2>&1 
!pip install "gym[box2d]" > /dev/null 2>&1 
!pip install "stable-baselines3[extra]" > /dev/null 2>&1 

!apt-get install x11-utils > /dev/null 2>&1 
!pip install pyglet > /dev/null 2>&1 
!apt-get install -y xvfb python-opengl > /dev/null 2>&1

!pip install pyvirtualdisplay > /dev/null 2>&1

!pip install plotly > /dev/null 2>&1

In [4]:
! wget http://www.atarimania.com/roms/Roms.rar
! mkdir /content/ROM/
! unrar e /content/Roms.rar /content/ROM/ -y
! python -m atari_py.import_roms /content/ROM/ > /dev/null 2>&1

--2021-10-12 18:50:34--  http://www.atarimania.com/roms/Roms.rar
Resolving www.atarimania.com (www.atarimania.com)... 195.154.81.199
Connecting to www.atarimania.com (www.atarimania.com)|195.154.81.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11128004 (11M) [application/x-rar-compressed]
Saving to: ‘Roms.rar’

Roms.rar            100%[===================>]  10.61M   466KB/s    in 24s     

2021-10-12 18:50:59 (449 KB/s) - ‘Roms.rar’ saved [11128004/11128004]


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/Roms.rar

Extracting  /content/ROM/HC ROMS.zip                                      36%  OK 
Extracting  /content/ROM/ROMS.zip                                         74% 99%  OK 
All OK


In [5]:
import torch
import random
import numpy as np

torch.manual_seed(10)
random.seed(10)
np.random.seed(10)

# Ambiente

O ambiente utilizado será o Enduro-v0, um ambiente [OpenAI Gym](https://gym.openai.com/envs/Enduro-v0/) de corrida.

[Gym](https://gym.openai.com/docs/) é um kit de ferramentas para desenvolver e comparar algoritmos de aprendizagem por reforço. Ele não faz suposições sobre a estrutura do seu agente e é compatível com qualquer biblioteca de computação numérica.

O estado do ambiente Enduro consiste em 210x160 pixels.Uma recompensa de +1 é dada para cada carro ultrapassado e -1 para cada carro que passa pelo agente (mas a recompensa mínima é 0).

O objetivo consiste em manobrar um carro de corrida no National Enduro, uma corrida de resistência de longa distância. O objetivo da corrida é passar um certo número de carros a cada dia. Isso permitirá que o jogador continue correndo no dia seguinte. O piloto deve evitar outros pilotos e ultrapassar 200 carros no primeiro dia e 300 carros em cada dia seguinte.

Conforme o tempo passa, a visibilidade também muda. Quando é noite no jogo, o jogador só pode ver as luzes traseiras dos carros que se aproximam. Com o passar dos dias, os carros também se tornarão mais difíceis de evitar. O clima e a hora do dia são fatores importantes para jogar. Durante o dia, o jogador pode dirigir por um trecho de gelo na estrada que limitaria o controle do veículo, ou um trecho de neblina pode reduzir a visibilidade.

[Descrição da Wikipedia](https://en.wikipedia.org/wiki/Enduro_%28video_game%29)

In [6]:
# Procedimento para renderizar o ambiente no Google Colab
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()

from matplotlib import pyplot as plt, animation
%matplotlib inline
from IPython import display

def create_anim(frames, dpi, fps):
    plt.figure(figsize=(frames[0].shape[1] / dpi, frames[0].shape[0] / dpi), dpi=dpi)
    patch = plt.imshow(frames[0])
    def setup():
        plt.axis('off')
    def animate(i):
        patch.set_data(frames[i])
    anim = animation.FuncAnimation(plt.gcf(), animate, init_func=setup, frames=len(frames), interval=fps)
    return anim

def display_anim(frames, dpi=72, fps=60):
    anim = create_anim(frames, dpi, fps)
    return anim.to_jshtml()

def save_anim(frames, filename, dpi=72, fps=50):
    anim = create_anim(frames, dpi, fps)
    anim.save(filename)

class trigger:
    def __init__(self):
        self._trigger = True

    def __call__(self, e):
        return self._trigger

    def set(self, t):
        self._trigger = t

Interagimos no ambiente através da função `step`, que nos retorna quatro valores: observação, recompensa, done, info. Esta é uma implementação do clássico “loop agente-ambiente”. A cada passo de tempo, o agente escolhe uma ação e o ambiente retorna uma observação e a recompensa.
<br>

<img src='https://drive.google.com/uc?id=1TXdjYkbfm2EvtCbVIpe5BkUgXJY1d1zE' width="600" height="250">

In [7]:
import gym
environment_id = "EnduroNoFrameskip-v4"       # Nome do ambiente utilizado

In [8]:
env = gym.make(environment_id)                # Criando o ambiente

frames = []
episodes = 1
for episode in range(1, episodes+1):
    obs = env.reset()                                   # Retorna a observação inicial
    done = False
    score = 0
    while not done:
        frames.append(env.render(mode='rgb_array'))     # Renderizando o ambiente
        action = env.action_space.sample()              # Seleciona uma ação aleatória
        obs, reward, done, info = env.step(action)      # Executa a ação selecionada
        score += reward
    print("\n\nEpisódio: {} Pontuação: {}".format(episode,score))
env.close()



Episódio: 1 Pontuação: 0.0


In [ ]:
display.HTML(display_anim(frames))

# Carregar Modelo Especialista

O modelo especialista que estamos disponibilizando para você é um agente de aprendizado por reforço treinado com o algoritmo Proximal Policy Optmization (PPO). Para isso, foi utilizado a biblioteca [Stable Baselines3](https://stable-baselines3.readthedocs.io/en/master/), que contém uma série de implementações de algoritmos de Aprendizado por Reforço em PyTorch.

In [9]:
!cd minicurso_rl && gdown --id 1ZV5fvCbU_gbTy1AraSR-ymNsiNQesBvt

Downloading...
From: https://drive.google.com/uc?id=1ZV5fvCbU_gbTy1AraSR-ymNsiNQesBvt
To: /content/gdrive/My Drive/minicurso_rl/EnduroNoFrameskip-v4.zip
100% 20.8M/20.8M [00:00<00:00, 57.0MB/s]


In [10]:
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.atari_wrappers import AtariWrapper
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecFrameStack

In [11]:
expert = PPO.load("minicurso_rl/EnduroNoFrameskip-v4")

Para criar o ambiente iremos aplicar alguns processamentos que irão ajudar o agente.

Os wrappers nos permitirão adicionar funcionalidade aos ambientes, como modificar observações e recompensas a serem fornecidas ao nosso agente. É comum na aprendizagem por reforço pré-processar as observações para torná-las mais fáceis de aprender. Um exemplo comum é ao usar entradas baseadas em imagem, para garantir que todos os valores estejam entre 0 e 1 ao invés de entre 0 e 255, como é mais comum com imagens RGB.

Para mais detalhes dos wrappers utilizados veja em: [Atari Wrappers](https://stable-baselines3.readthedocs.io/en/master/common/atari_wrappers.html) e [Vectorized Environments](https://stable-baselines3.readthedocs.io/en/master/guide/examples.html?highlight=make_vec_env#multiprocessing-unleashing-the-power-of-vectorized-environments).

In [12]:
env = make_vec_env(environment_id, wrapper_class=AtariWrapper)  
env = VecFrameStack(env, 4)

Vamos ver o quão bem o especialista consegue se sair no ambiente.

In [ ]:
mean_reward, std_reward = evaluate_policy(expert, env, n_eval_episodes=10)
print(f"Recompensa média = {mean_reward} +/- {std_reward}")

Recompensa média = 1128.1 +/- 122.71385414858422


In [ ]:
frames = []
episodes = 1
for episode in range(1, episodes+1):
    obs = env.reset()     # Retorna a observação inicial
    done = False
    score = 0
    while not done:
        frames.append(env.render(mode='rgb_array'))         # Renderizando o ambiente
        action = expert.predict(obs, deterministic=True)    # Seleciona uma ação do agente especialista
        obs, reward, done, info = env.step(action)          # Executa a ação selecionada
        score += reward
    print("\n\nEpisódio: {} Pontuação: {}".format(episode,score))
env.close()

display.HTML(display_anim(frames))

Agora, deixamos nosso especialista interagir com o ambiente e armazenar as observações e ações de especialistas resultantes para construir um conjunto de dados.

In [13]:
from tqdm import tqdm

In [14]:
num_interactions = int(4e4)

In [ ]:
if isinstance(env.action_space, gym.spaces.Box):
    expert_observations = np.empty((num_interactions,) + env.observation_space.shape)
    expert_actions = np.empty((num_interactions,) + (env.action_space.shape[0],))
else:
    expert_observations = np.empty((num_interactions,) + env.observation_space.shape)
    expert_actions = np.empty((num_interactions,) + env.action_space.shape)

# HW: Interaja  com o ambiente `env` conforme visto anteriormente. Armazene 
# as observações e as ações do especialista em `expert_observations` e 
# `expert_actions` respectivamente para construir o dataset.

#frames = []
episodes = 1
i = 0
while num_interactions > i:
#for episode in range(1, episodes+1):
    obs = env.reset()                                       # Retorna a observação inicial
    done = False
    score = 0
    while not done and num_interactions > i:
        #frames.append(env.render(mode='rgb_array'))        # Renderizando o ambiente
        action = expert.predict(obs, deterministic=True)    # Seleciona uma ação do agente especialista
        expert_observations[i] = obs
        expert_actions[i] = action[0][0]
        obs, reward, done, info = env.step(action)          # Executa a ação selecionada
        score += reward
        i+=1
    print("\n\nEpisódio: {} Pontuação: {}".format(episodes,score))
    episodes += 1
env.close()

# Salva os dados (observação, ação)
np.savez_compressed(
    "minicurso_rl/expert_data",
    expert_actions=expert_actions,
    expert_observations=expert_observations,
)

In [22]:
try:
    expert_observations, expert_actions
except NameError:
    pass
else:
  del expert_observations, expert_actions

# Carrega os dados salvos
data = np.load("minicurso_rl/expert_data.npz")

- Para usar perfeitamente o PyTorch no processo de treinamento, criamos uma subclasse de `ExpertDataset` do `Dataset` base do Pytorch
- Observe que inicializamos o conjunto de dados com as observações e ações de especialistas geradas anteriormente.
- Implementamos ainda as [funções mágicas](https://rszalski.github.io/magicmethods/) `__getitem__` e` __len__` do Python para permitir que o manuseio do conjunto de dados do PyTorch acesse linhas arbitrárias no conjunto de dados e informá-lo sobre o comprimento do conjunto de dados.
- Para obter mais informações sobre os conjuntos de dados de PyTorch, você pode ler: https://pytorch.org/docs/stable/data.html.

In [23]:
from torch.utils.data.dataset import Dataset, random_split

In [24]:
class ExpertDataSet(Dataset):
    def __init__(self, expert_observations, expert_actions):
        self.observations = expert_observations
        self.actions = expert_actions
        
    def __getitem__(self, index):
        return (self.observations[index], self.actions[index])

    def __len__(self):
        return len(self.observations)

Agora instanciamos o `ExpertDataSet` e o dividimos em conjuntos de dados de treinamento e teste.

In [25]:
expert_dataset = ExpertDataSet(data["expert_observations"], data["expert_actions"])

del data

train_size = int(0.8 * len(expert_dataset))     # 80% dos dados para treinamento
test_size = len(expert_dataset) - train_size    # E o restante dos dados para teste

train_expert_dataset, test_expert_dataset = random_split(
    expert_dataset, [train_size, test_size]
)

In [26]:
print("# test_expert_dataset: ", len(test_expert_dataset))
print("# train_expert_dataset: ", len(train_expert_dataset))

# test_expert_dataset:  8000
# train_expert_dataset:  32000


# Treinar o agente estudante

Nossos próximos passos:

1. Extraímos a rede de políticas de nosso aluno.
2. Carregamos o conjunto de dados de especialistas (rotulados) contendo observações de especialistas como entradas e ações de especialistas como alvos.
3. Realizamos aprendizagem supervisionada, ou seja, ajustamos os parâmetros da rede de políticas de forma que, dadas as observações de especialistas como entradas para a rede, suas saídas correspondam aos alvos (ações de especialistas).


Ao treinar a rede de políticas dessa maneira, o agente aluno correspondente é ensinado a se comportar como o agente especialista que foi usado para criar o conjunto de dados especialista (Behavior Cloning).

In [27]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions.categorical import Categorical
from torch.optim.lr_scheduler import StepLR

In [28]:
batch_size=128
epochs=20
scheduler_gamma=0.99
learning_rate=5e-3
log_interval=100
no_cuda=False    
seed=1
test_batch_size=128

In [29]:
use_cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

Agora iremos definir a rede neural que iremos utilizar para o aluno estudante. Aqui criamos um agente de aprendizado por reforço, e extraimos dele a rede da política. Alternativamente você pode construir a sua própria rede neural.

Como estamos utilizando imagens como entrada, iremos utilizar uma rede chamada de Rede Neural Convolucional (Convolutional Neural Network - CNN).

<br>

- https://towardsdatascience.com/pytorch-basics-how-to-train-your-neural-net-intro-to-cnn-26a14c2ea29

- https://medium.com/swlh/introduction-to-cnn-image-classification-using-cnn-in-pytorch-11eefae6d83c

- https://www.analyticsvidhya.com/blog/2019/10/building-image-classification-models-cnn-pytorch/


In [30]:
from torchsummary import summary

student = PPO('CnnPolicy', env, verbose=1)

# Extrair politica inicial
model = student.policy.to(device)

# Mostra um sumário da rede, mostrando todas as suas camadas 
summary(model, (4, 84, 84))

Using cuda device
Wrapping the env in a VecTransposeImage.
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 20, 20]           8,224
              ReLU-2           [-1, 32, 20, 20]               0
            Conv2d-3             [-1, 64, 9, 9]          32,832
              ReLU-4             [-1, 64, 9, 9]               0
            Conv2d-5             [-1, 64, 7, 7]          36,928
              ReLU-6             [-1, 64, 7, 7]               0
           Flatten-7                 [-1, 3136]               0
            Linear-8                  [-1, 512]       1,606,144
              ReLU-9                  [-1, 512]               0
        NatureCNN-10                  [-1, 512]               0
     MlpExtractor-11     [[-1, 512], [-1, 512]]               0
           Linear-12                    [-1, 1]             513
           Linear-13                    [-1,

Como visto em aula, queremos minimizar a diferença entre a resposta correta e a resposta do modelo. A primeira tarefa é, portanto, definir um critério que mede o erro entre cada elemento na entrada x e no destino y.

Aqui precisamos nos atentar em alguns pontos. 

Box e Discrete são os dois tipos de espaço mais comumente usados para representar os espaços de Observação e Ação em ambientes do Gym. 

- Box: Uma caixa dimensional, onde cada coordenada fica entre um limite definido por [baixo, alto]
- Discrete: O espaço consiste em n pontos distintos, cada um mapeado para um valor inteiro no intervalo [0, n-1]


No caso do ambiente Enduro,as ações são discretas, onde será selecionado um valor entre 0 e n-1 para ser aplicado ao ambiente.

As saídas da rede é uma lista de probabilidade de selecionar cada uma dessas ações. Iremos executar a ação com a maior probabilidade dada pela rede.

<img src='https://drive.google.com/uc?id=1KEBtAKI5kOAC7PfcK3SRQIdE1sQwmAza' width="550" height="180">

Como iremos definir o erro da entrada e do destino?

O que queremos minimizar é a distância entre duas distribuições de probabilidade - prevista e real.

Considere um classificador que prediz se um dado animal é um cão, gato ou cavalo com uma probabilidade associada a cada um. 

Suponha que a imagem original seja de um cachorro e o modelo preveja 0.2, 0.7, 0.1 como probabilidade para três classes em que as probabilidades verdadeiras se parecem com 1, 0, 0. O que desejamos idealmente é que nossas probabilidades previstas sejam próximas às originais. Portanto, precisamos nos certificar de que estamos minimizando a diferença entre as duas probabilidades.

Para isso temos uma loss chamada de Cross-Entropy que nos ajuda a calcular essa diferença. Veja mais em: https://towardsdatascience.com/cross-entropy-loss-function-f38c4ec8643e

In [31]:
nb_actions = env.action_space.n
print("O número total de ações possíveis é: ", nb_actions)

O número total de ações possíveis é:  9


In [32]:
# HW: Implementar função de Loss
criterion = nn.CrossEntropyLoss()

In [33]:
# HW: Implementar função de Acurácia
def acc(model_out, true_out):
    model_a = model_out.argmax(1)
    true_a = true_out
    return torch.sum(model_a == true_a)

In [35]:
# HW: Implementar função de Treino
# ela deve retornar informações de loss e acurácia
def train():
    _loss = 0.0
    _acc = 0.0

    model.train()
    for data, target in train_loader:
        data = data.permute(0, 3, 1, 2)
        data, target = data.to(device), target.to(device)

        if isinstance(env.action_space, gym.spaces.Box):
            # A2C/PPO policy outputs actions, values, log_prob
            action, _, _ = model(data)
            action_prediction = action.double()
        else:
            # Retrieve the logits for A2C/PPO when using discrete actions
            latent_pi, _, _ = model._get_latent(data)
            logits = model.action_net(latent_pi)
            action_prediction = logits
            target = target.long()

        train_loss = criterion(action_prediction, target)
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        _loss += train_loss.data.cpu().numpy()
        _acc += acc(action_prediction, target).data.cpu().numpy()
    
    _loss /= float(len(train_loader.dataset))
    _acc /= float(len(train_loader.dataset))
    print(f"Train Set: Loss {_loss:.4f} \tAccuracy {_acc*100:.2f} %")
    return _loss, _acc

In [36]:
def test():
    _loss = 0.0
    _acc = 0.0

    model.eval()
    with torch.no_grad():
        for data, target in test_loader:
            data = data.permute(0, 3, 1, 2)
            data, target = data.to(device), target.to(device)

            if isinstance(env.action_space, gym.spaces.Box):
                # A2C/PPO policy outputs actions, values, log_prob
                action, _, _ = model(data)
                action_prediction = action.double()
            else:
                # Retrieve the logits for A2C/PPO when using discrete actions
                latent_pi, _, _ = model._get_latent(data)
                logits = model.action_net(latent_pi)
                action_prediction = logits
                target = target.long()
            
            test_loss = criterion(action_prediction, target)
            _loss += test_loss.data.cpu().numpy()
            _acc += acc(action_prediction, target).data.cpu().numpy()

    _loss /= float(len(test_loader.dataset))
    _acc /= float(len(test_loader.dataset))
    print(f"Test Set: Loss {_loss:.4f} \tAccuracy {_acc*100:.2f} %")
    return _loss, _acc    

Avalie o agente antes do treinamento (seu comportamento deve ser aleatório)

In [37]:
mean_reward, std_reward = evaluate_policy(student, env, n_eval_episodes=10)
print(f"Recompensa média = {mean_reward} +/- {std_reward}")

Recompensa média = 110.0 +/- 330.0


In [38]:
# Aqui, usamos PyTorch `DataLoader` para carregar o `ExpertDataset` criado anteriormente para treinamento e teste
train_loader = torch.utils.data.DataLoader(
    dataset=train_expert_dataset, batch_size=batch_size, shuffle=True, **kwargs
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_expert_dataset, batch_size=test_batch_size, shuffle=True, **kwargs,
)

# Defina um Otimizador e uma programação de taxa de aprendizagem (learning rate).
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=1, gamma=scheduler_gamma)

Tendo definido o procedimento de treinamento, podemos agora executar o treinamento!

In [39]:
# Agora estamos finalmente prontos para treinar o modelo de política.
train_loss, train_acc = [], []
test_loss, test_acc = [], []
_learning_rate = []

for epoch in range(1, epochs + 1):
    _learning_rate.append(scheduler.get_lr()[0])
    print("learning rate: ", scheduler.get_lr()[0])

    _train_loss, _train_acc = train()
    _test_loss, _test_acc = test()

    train_loss.append(_train_loss)
    train_acc.append(_train_acc)
    test_loss.append(_test_loss)
    test_acc.append(_test_acc)
    
    scheduler.step()

learning rate:  0.005


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


Train Set: Loss 0.0088 	Accuracy 57.71 %
Test Set: Loss 0.0067 	Accuracy 68.08 %
learning rate:  0.0049005
Train Set: Loss 0.0061 	Accuracy 69.91 %
Test Set: Loss 0.0057 	Accuracy 71.20 %
learning rate:  0.004851495
Train Set: Loss 0.0053 	Accuracy 73.24 %
Test Set: Loss 0.0053 	Accuracy 73.24 %
learning rate:  0.00480298005
Train Set: Loss 0.0047 	Accuracy 76.12 %
Test Set: Loss 0.0050 	Accuracy 74.99 %
learning rate:  0.0047549502495
Train Set: Loss 0.0043 	Accuracy 78.35 %
Test Set: Loss 0.0052 	Accuracy 74.69 %
learning rate:  0.004707400747005
Train Set: Loss 0.0038 	Accuracy 80.50 %
Test Set: Loss 0.0051 	Accuracy 75.20 %
learning rate:  0.00466032673953495
Train Set: Loss 0.0035 	Accuracy 82.29 %
Test Set: Loss 0.0054 	Accuracy 75.29 %
learning rate:  0.0046137234721396
Train Set: Loss 0.0031 	Accuracy 84.10 %
Test Set: Loss 0.0056 	Accuracy 75.40 %
learning rate:  0.004567586237418204
Train Set: Loss 0.0028 	Accuracy 85.72 %
Test Set: Loss 0.0058 	Accuracy 76.68 %
learning rate

In [40]:
import plotly.graph_objs as go

In [41]:
fig = go.Figure([
    go.Scatter(
        y=train_acc,
        x=[ep for ep in range(1, epochs + 1)],
        mode='lines',
        name="Acurácia de Treino"
    ),
    go.Scatter(
        y=test_acc,
        x=[ep for ep in range(1, epochs + 1)],
        mode='lines',
        name="Acurácia de Teste"
    ),
])
fig.update_layout(
    title="Acurácia",
    yaxis = dict(
        tickformat = "%",
    ),
    xaxis = dict(
        title = "Época",
    )
)
fig.show()

In [42]:
fig = go.Figure([
    go.Scatter(
        y=train_loss,
        x=[ep for ep in range(1, epochs + 1)],
        mode='lines',
        name="Loss de Treinamento"
    ),
    go.Scatter(
        y=test_loss,
        x=[ep for ep in range(1, epochs + 1)],
        mode='lines',
        name="Loss de Teste"
    ),
])
fig.update_layout(
    title="Loss",
    xaxis = dict(title="Época")
)
fig.show()

In [43]:
fig = go.Figure([
    go.Scatter(
        y=_learning_rate,
        x=[ep for ep in range(1, epochs + 1)],
        mode='lines',
        name="Learning Rate"
    ),
])
fig.update_layout(
    title="Learning Rate",
    xaxis = dict(title="Época")
)
fig.show()

Finalmente, vamos testar o quão bem nosso aluno aprendeu a imitar o comportamento do especialista

In [44]:
# Inserir a rede treinada de volta no agente estudante
student.policy = model

In [45]:
mean_reward, std_reward = evaluate_policy(student, env, n_eval_episodes=10)
print(f"Mean reward = {mean_reward} +/- {std_reward}")

Mean reward = 169.9 +/- 46.39493506838866


In [46]:
frames = []
episodes = 1
for episode in range(1, episodes+1):
    obs = env.reset()     
    done = False
    score = 0
    while not done:
        frames.append(env.render(mode='rgb_array'))           
        action = student.predict(obs, deterministic=True)   
        obs, reward, done, info = env.step(action)       
        score += reward
    print("\n\nEpisódio: {} Pontuação: {}".format(episode,score))
env.close()



Episódio: 1 Pontuação: [171.]


In [ ]:
display.HTML(display_anim(frames))

# Bônus

O algoritimo Dagger é um algoritimo interativo que aproxima as distribuições de trajetórias de alunos e especialistas ao rotular pontos de dados adicionais resultantes da aplicação da política atual.

Em sua forma mais simples, o algoritmo procede da seguinte maneira. Na primeira iteração, ele usa a política do especialista para reunir um conjunto de dados de trajetórias $D$ e treinar uma política $\pi_{2}$ que melhor imita o especialista nessas trajetórias. Então, na iteração $n$, ele usa $\pi_{n}$ para coletar mais trajetórias e adiciona essas trajetórias ao conjunto de dados $D$. A próxima política $\pi_{n+1}$ é a que melhor imita o especialista em todo o conjunto de dados $D$.

É como se a cada passo, perguntássemos ao especialista sua opinião sobre nossa trajetória atual. Em seguida, reunindo esta opinião (sua resposta aos estados que encontramos) e os conjuntos de dados anteriores de trajetórias,
podemos treinar uma nova política mais precisa porque estamos levando em consideração a opinião de mais especialistas.

<br>

Algoritmo [Dagger](http://proceedings.mlr.press/v15/ross11a/ross11a.pdf):

<img src='https://drive.google.com/uc?id=1rMERL80AGmDRR0fKVfq0KjhJjGPt4YKt' width="450" height="250">


A tarefa bônus consistirá em implementar o algoritmo Dagger.
